In [1]:
import pandas as pd
import xlsxwriter
import numpy as np
import os
import time
import warnings
warnings.filterwarnings('ignore')
startTime = time.time()

pd.set_option("display.max_columns", 100) #local settings


# reads sheets for mapping
data_dict = pd.read_excel('./internal_files/internal_LOC.xlsx', sheet_name=None)
supp_list = data_dict['SuppIDs']
status_list = data_dict['Status']
po_list = data_dict['PO#']
pa_list = data_dict['PA#']
client_list = data_dict['ClientPM']

# empty DFs
cons_types = pd.DataFrame()
consolidated_files = pd.DataFrame()
file_name = pd.DataFrame()
full_names = pd.DataFrame()

path = 'analyses'
folder = [file for file in os.listdir(path)] #reads the "analyses" folder

for file in folder: 
    Per = pd.concat(pd.read_excel('./analyses/'+file, sheet_name=['Per_File']), ignore_index=True) #reads Per_File sheet
    Per = Per.drop_duplicates(subset = ['Language Tag', 'Project Name'], keep = 'last') #droping duplicated PIDs
    Per['Project Number'] = Per['Project Number'].astype('str')
    Total = pd.concat(pd.read_excel('./analyses/'+file, sheet_name=['Total']), ignore_index=True) #reads Total sheet
    Per_new = Per.drop(['Translation Type', 'Project Name', 'Language', 'Name', 'ICE', '100% Matches', '99-95% Matches', '94-85% Matches', '84-75% Matches', '74-0% Matches','Reps', 'Total', 'Adjusted Volume', 'Review Volume', 'Date Due', 'Workgroup'], axis = 1) #deleting unnecessary columns from Per dataframe
    Total_new = Total.drop(['Total', 'Adjusted Volume', 'Review Volume', 'Language Tag'], axis=1) #deleting unnecessary columns from Total dataframe
    
    #reads all file names and adds them to a separate data frame in form: "AWS-HO#_Projectname"
    f_name = pd.concat(pd.read_excel(path+'\\'+file, sheet_name=['Total']), ignore_index=True) #adding file name column "Project Name"
    f_name['Project Name'] = file #new column
    file_name = pd.concat([f_name], axis=1) #creating a dataframe
    file_name = file_name.drop(['Language Tag', 'Language', 'Translation Type', 'ICE', '100% Matches', '99-95% Matches', '94-85% Matches', '84-75% Matches', '74-0% Matches', 'Reps', 'Total', 'Adjusted Volume', 'Review Volume'], axis = 1) #deleting unnecessary columns from new dataframe (with project names)
    full_names = pd.concat([file_name, full_names], axis=0, ignore_index=True) #creating fresh dataframe with all file names
    
    #removes duplicates in PIDs and adds 'PID' string at the very beggining
    Per_new = pd.pivot_table(Per_new, index=['Language Tag'], values='Project Number',aggfunc=lambda x: ', '.join(x)) #adding a separator ", " between all PID numbers
    Per_new['Project Number'] = 'PID' + Per_new['Project Number'] 
    Per_new.reset_index(inplace=True)
    
    #creates a consolidated dataframe with the combined data above
    consolidated_sheets = pd.concat([Per_new, Total_new], axis=1)[['Language Tag', 'Project Number', 'Language', 'Translation Type','ICE', '100% Matches', '99-95% Matches', '94-85% Matches', '84-75% Matches', '74-0% Matches', 'Reps']] #creating a consolidated dataframe with Per and Total
    consolidated_sheets = consolidated_sheets.iloc[::-1] #changing an order of dataframe values (reverse a dataframe)
    consolidated_files = pd.concat([consolidated_sheets, consolidated_files], axis=0, ignore_index=True)
    consolidated_files.reset_index(drop=True, inplace=True) #reseting indexes for ignore NaNs
    consolidated_files_up = pd.concat([full_names, consolidated_files], axis=1, ignore_index=True)
    
consolidated_files.reset_index(drop=True, inplace=True)
# Reverse the order of rows in consolidated_files_up
consolidated_files_up = consolidated_files_up.iloc[::-1]
# Reset the index of consolidated_files_up and assign to consolidated_files_up
consolidated_files_up.reset_index(drop=True, inplace=True)
# Rename the columns of consolidated_files_up
consolidated_files_up.columns = ['File name', 'Language Tag', 'PIDs', 'Language', 'Translation type', 'ICE', '100% Matches', '99-95% Matches', '94-85% Matches', '84-75% Matches', '74-0% Matches', 'Reps']


# Split the 'File name' column of consolidated_files_up and create two new columns
consolidated_files_up[['AWS-HO', 'Project name']] = consolidated_files_up['File name'].str.split('_', 1, expand=True)
# Convert the 'Project name' column to string type and remove '.xlsx' extension
consolidated_files_up['Project name'] = consolidated_files_up['Project name'].astype(str).str.rstrip('.xlsx')
# Drop the 'File name' and 'Language Tag' columns from consolidated_files_up
consolidated_files_up.drop(['File name', 'Language Tag'], axis=1, inplace=True)
# Reorder the columns in consolidated_files_up
consolidated_files_up = consolidated_files_up[['AWS-HO', 'PIDs', 'Project name', 'Language', 'Translation type', 'ICE', '100% Matches', '99-95% Matches', '94-85% Matches', '84-75% Matches', '74-0% Matches', 'Reps']]
# Rename the columns of consolidated_files_up
consolidated_files_up.columns = ['AWS-HO', 'Project Number', 'Project name', 'Language', 'Translation type', 'ICE', '100% Matches', '99-95% Matches', '94-85% Matches', '84-75% Matches', '74-0% Matches', 'Reps']
# final_details
final_details = consolidated_files_up


#reads all Per_File tabs from the "analyses" folder, takes the oldest date from "Date Due" column and adds it to a separate Dataframe below: (cons_dates) 

cons_dates = pd.DataFrame()
cons_dates2 = pd.DataFrame()

# reads multiple "Total" and "Per" sheets
for file in folder:
    # Read Per_File and Total tabs
    Per = pd.read_excel('./analyses/'+file, sheet_name='Per_File')
    
    # concatenate all Per tab vertically
    total_dates = pd.concat([Per]) 

    # extract the 'Date Due' column and convert to datetime
    dates = total_dates[['Date Due']]
    dates['Date Due'] = pd.to_datetime(dates['Date Due'], errors='coerce')

    # sort by 'Date Due' and append to cons_dates dataframe
    dates = dates.sort_values(by='Date Due', ascending=False).iloc[:1]
    cons_dates = pd.concat([cons_dates, dates], axis=0).reset_index(drop=True)

    # extract the 'Date Created' column and convert to datetime
    creation_dates_list = Per[['Date Created']].iloc[:1].copy()
    creation_dates_list['Date Created'] = pd.to_datetime(creation_dates_list['Date Created'])
    creation_dates_list.sort_values(by='Date Created', ascending=False, inplace=True)
    cons_dates3 = pd.concat([cons_dates2, creation_dates_list.rename(columns={'Date Created': 'Creation Date'})], ignore_index=True)

# PAYMENT TAB
payment = final_details[['AWS-HO', 'Project name']]

# PROJECT DETAILS tab
# dashboard with columns on Project Details tab. We can add/edit columns and their values in the list below


# Pa pre-built structure for future PAYMENT tab: 
final_payment = payment[['AWS-HO', 'Project name']].drop_duplicates()

# PAYMENT TAB; dashboard
final_payment['Client PO Number'] = ''
final_payment['HO Number'] = final_details['AWS-HO']
final_payment['Translation Type'] = ''
final_payment['Purchase Month'] = '2023_02'
final_payment['Billing Month'] = '2023_02'
final_payment['Client PM Name'] = ''
final_payment.reset_index(drop=True, inplace=True)

# 
sep_list = final_payment['Project name'].str.split('-', 4, expand=True)
sep_list.columns = ['first', 'second', 'third', 'fourth', 'drop1']
sep_list = sep_list[['first', 'second', 'third', 'fourth']]
sep_list['Project name'] = sep_list.apply(lambda x: '-'.join(x.dropna().astype(str)), axis=1)
sep_list = sep_list[['Project name', 'first','second', 'third', 'fourth']]

sep_list2 = sep_list[['Project name']]


# extracting project names based on specific patterns:
sep_list3 = sep_list.drop(['first', 'second', 'third', 'fourth'], axis=1) #project names, but only with 4 "_" separators
sep_list3.loc[sep_list3["Project name"].str.contains("APSF")] = sep_list2.loc[sep_list2["Project name"].str.contains("APSF")].applymap(lambda x: "-".join(x.split("-")[2:4]))
sep_list3.loc[sep_list3["Project name"].str.contains("AWS-CMS|AWS-DIR|AWS-LCM|AWS-BLOG|AWS-KB|AWS-SD")] = sep_list2.loc[sep_list2["Project name"].str.contains("AWS-CMS|AWS-DIR|AWS-LCM|AWS-BLOG|AWS-KB|AWS-SD")].applymap(lambda x: "-".join(x.split("-")[1:3]))
sep_list3.loc[sep_list3["Project name"].str.contains("AWS-SW-CON")] = sep_list2.loc[sep_list2["Project name"].str.contains("AWS-SW-CON")].applymap(lambda x: "-".join(x.split("-")[1:3]))

# exceptions; in some projects there is no separator "-", when it should normally be, fix such cases here:
# (you can compare sep_list2 & sep_list3 data frames for more visibility)
sep_list3.loc[sep_list3['Project name'].str.contains('APSF'), 'Project name'] = 'CON-APSF'
sep_list3.loc[sep_list3['Project name'].str.contains('SW-CON-'), 'Project name'] = 'SW-CON'
sep_list3.loc[sep_list3['Project name'].str.contains('Soft', case=False, regex=True), 'Project name'] = 'SW-CON'
sep_list3.loc[sep_list3['Project name'].str.contains('CMS-KB'), 'Project name'] = 'KB-CQClient'

# PO number vlookup by sep_list3 values and dictionary (check folder internal_final -> internal_loc.xlsx)
po_numbers_final = pd.merge(sep_list3, po_list, on='Project name', how='left').drop('Project name', axis=1)
po_numbers_final.reset_index(drop=True, inplace=True)

# PA number vlookup by sep_list3 values and dictionary
pa_numbers_final = pd.merge(sep_list3, pa_list, on='Project name', how='left').drop('Project name', axis=1)
pa_numbers_final.reset_index(drop=True, inplace=True)

# Merge the PA and PO number columns with final_payment DataFrame
final_payment = pd.concat([pa_numbers_final, po_numbers_final, final_payment], axis=1)

# code uses sep_list3 for PA# & PO# columns, but sep_list4 will be used for content categories only
sep_list4 = final_payment[['Project name']].copy().dropna()
sep_list4.loc[sep_list4["Project name"].str.contains("SW-CON-APSF")] = sep_list4.loc[sep_list4["Project name"].str.contains("SW-CON-APSF")].applymap(lambda x: "-".join(x.split("-")[3:4]))
sep_list4.loc[sep_list4["Project name"].str.contains("AWS")] = sep_list4.loc[sep_list4["Project name"].str.contains("AWS")].applymap(lambda x: "-".join(x.split("-")[1:4]))
sep_list4.loc[sep_list4["Project name"].str.contains("CMS|BLOG|DIR|SD")] = sep_list4.loc[sep_list4["Project name"].str.contains("CMS|LCM|DIR|SD")].applymap(lambda x: "-".join(x.split("-")[0:2]))

# Content category exceptions; in some projects there is no separator "-", when it should normally be, fix such cases here:
sep_list4.loc[sep_list4['Project name'].str.contains('APSF'), 'Project name'] = 'SW-CON-APSF'
sep_list4.loc[sep_list4['Project name'].str.contains('WAGEngine.S3'), 'Project name'] = 'SW-CON-WAGEngine.S3'
sep_list4.loc[sep_list4['Project name'].str.contains('AWSActivate.S3'), 'Project name'] = 'SW-CON-AWSActivate.S3'
sep_list4.loc[sep_list4['Project name'].str.contains('Aperture.S3'), 'Project name'] = 'SW-CON-Aperture.S3'

### sep_list4['Project name'] = sep_list4['Project name'].str.replace('.', '-') mb use this one above no avoid exceptions

# move generated content categories to main DF
final_payment['Content Category'] = sep_list4['Project name']

#ClientPM column:
pm_templist = pd.DataFrame(final_payment['PO#'])
# Merge client_list and final_payment on PO# to get ClientPM
final_payment = pd.merge(final_payment, client_list, on='PO#', how='left')

#############
langs_count = final_details['AWS-HO'].value_counts().sort_index().reset_index()
langs_count = langs_count.rename(columns={'index': 'AWS-HO', 'AWS-HO': '# of Langs'})
final_payment = pd.concat([langs_count, final_payment], axis=1) #adding #ofLangs column ("Langs")
#########

final_details = final_details[['AWS-HO', 'Project name', 'Project Number', 'Language', 'Translation type', 'ICE', '100% Matches', '99-95% Matches', '94-85% Matches', '84-75% Matches', '74-0% Matches', 'Reps']]

# Dataframe with Translation Types for Payment tab. 
# It reads the number of translation types from the analysis and returns the most common value. 
# In the "count" column we can see how many types of HT/MTPE-P are in analyses.
# if the nubmer of HT type exceeds MTPE type = HT will be selected; the same in wise versa:
for file in folder:
    # Dataframe with Translation Types for Payment tab. "30" is the largest value of rows that can be in the analysis. if AWS division will add a big projects that contains > 30langs. on Total - update this number.
    Total_temp = pd.read_excel('./analyses/' + file)
    tran_type = Total_temp['Translation Type'].value_counts().iloc[:30].reset_index(name='count')
    tran_type = tran_type.drop_duplicates(subset='index').head(1)  # Drop duplicates and keep first row
    tran_type = pd.concat([tran_type], axis=1)
    cons_types = pd.concat([cons_types, tran_type], axis=0)  # this dataframe consist the most frequency translation type and keeping it as the last value + column with a sum of them
    final_types = cons_types.drop('count', axis=1).reset_index(drop=True)
    
    
# PAYMENT dashboard #2; 
final_payment['Client PO Number'] = po_numbers_final['PO#']
final_payment['Handoff Date'] = cons_dates3['Creation Date'] 
final_payment['Handoff Date'] = final_payment['Handoff Date'].dt.strftime('%d/%m/%Y')
final_payment['Requested Delivery'] = pd.to_datetime(final_payment['Handoff Date']).apply(pd.DateOffset(1))
final_payment['Translation Type'] = final_types['index']
final_payment['# of Langs'] = langs_count['# of Langs']

final_payment = final_payment[['Project name', 'PA#', 'Client PO Number', 'HO Number', '# of Langs', 'Content Category', 'Translation Type', 'Purchase Month', 'Billing Month', 'ClientPM', 'Requested Delivery']]
#PAYMENT row insert
#inserts empty rows at the position where the number is missed from the continuous sequence of column "AWS-HO"
# in other words: if xlsx in missing from the folder - inserts empty row + adds hand off in the column
final_payment['Number_payment'] = final_payment['HO Number'].str.lstrip('AWS-HO').astype(int)
# Insert missing rows
min_number = final_payment['Number_payment'].min()
max_number = final_payment['Number_payment'].max()
missing_numbers = set(range(min_number, max_number+1)) - set(final_payment['Number_payment'])
missing_rows = pd.DataFrame({'Number_payment': list(missing_numbers)})
final_payment = pd.concat([final_payment, missing_rows])
final_payment.sort_values(by='Number_payment', inplace=True)
final_payment.reset_index(drop=True, inplace=True)
# Convert number back to HO Number format
final_payment['HO Number'] = 'AWS-HO' + final_payment['Number_payment'].astype(str)
final_payment.drop(['Number_payment'], axis=1, inplace=True)
final_payment = final_payment[['HO Number'] + list(final_payment.columns[:-1])]

#final_payment = final_payment[['Project name', 'PA#', 'Client PO Number', 'HO Number', '# of Langs', 'Content Category', 'Translation Type', 'Purchase Month', 'Billing Month', 'ClientPM', 'Handoff Date', 'Handoff Time (CET)', 'Requested Delivery', 'Delivered']]
# PROJ. DETAILS row insert
# Project details tab; does the same (insert rows) but different logic:
final_details['Test_column2'] = final_details['AWS-HO'].map(lambda x: x.lstrip('AWS-HO')).astype('int')
final_details.set_index("Test_column2")
second_column = final_details.pop('Test_column2')
final_details.insert(0, 'Number_projdetails', second_column)
final_details['Number_projdetails'] = final_details['Number_projdetails'].astype('int')
final_details = final_details.merge(how='right', on='Number_projdetails', right = pd.DataFrame({'Number_projdetails':np.arange(final_details.iloc[0]['Number_projdetails'],
                                                                                                       final_details.iloc[-1]['Number_projdetails']+1,1)})).reset_index().drop(['index'], axis=1)
final_details['Number_projdetails'] = 'AWS-HO' + final_details['Number_projdetails'].astype(str)
final_details.rename(columns={'Number_projdetails': 'HO Number'}, inplace=True)
final_details = final_details.drop('AWS-HO', axis=1)

# tickets without any analysis will be put on hold status, while those with analysis will have the 'MTPE-P/HT' status
final_payment['Status'] = np.nan
final_payment['Status'] = np.where(final_payment['Project name'].isna(), 'On Hold',
                                        'MTPE-P / HT')

# Set purchase and billing months to '2023_03' if project name is missing
default_month = '2023_03'
final_payment['Purchase Month'] = final_payment['Billing Month'] = np.where(final_payment['Project name'].isna(), 
                                                                            default_month, default_month)

final_payment['Project name'] = final_payment['Project name'].str.replace('.XLSX', '', regex=True)
final_details['Project name'] = final_details['Project name'].str.replace('.XLSX', '', regex=True)

# deletes substrings in project names:
repl1 = ['_edited', '_updated', '.XLSX']
for string in repl1:
    final_payment['Project name'] = final_payment['Project name'].str.replace(string, '', case=False, regex=True)

# Replace 'DIR-' with 'CMS-' in the 'Content_Category' column
final_payment['Content Category'] = final_payment['Content Category'].str.replace('DIR-', 'CMS-', case=False)

# ".TRA" replacements (TaC projects)
final_payment['Content Category'] = np.where((final_payment['Project name'].notnull()) &
                                             final_payment['Project name'].fillna('').str.contains('.TRA'), 'CMS-TRA',
                                             final_payment['Content Category'])
final_payment['Client PO Number'] = np.where((final_payment['Project name'].notnull()) & 
                                             (final_payment['Project name'].fillna('').str.contains('.TRA')), '3M-07336998 (TaC)',
                                             final_payment['Client PO Number'])
final_payment['ClientPM'] = np.where((final_payment['Project name'].notnull()) &
                                     final_payment['Project name'].fillna('').str.contains('.TRA'), 'Monhem Giumaa',
                                     final_payment['ClientPM'])


# General -> SME replacements. adds coment for specific projects; need to update manually in the tracker
sme = [
    '.TRA', '-PSM', '-PSCP'
]
final_payment['Discrepancies'] = np.where((final_payment['Project name'].notnull()) &
                                          final_payment['Project name'].str.contains('|'.join(sme)), 'Change "General" on "SME" for this project in the tracker',
                                          None)
final_details['Language'] = np.where(final_details['Language'] == 'ENU', 'CHECK', final_details['Language'])


# dtypes
final_payment = final_payment.astype({
    'Project name': 'string',
    'Client PO Number': 'string',
    'HO Number': 'string',
    'Content Category': 'string',
    'Translation Type': 'string',
    'ClientPM': 'string'
})

final_payment = final_payment[['Project name', 'PA#', 'Client PO Number', 'HO Number', '# of Langs', 'Content Category', 'Translation Type', 'Status', 'Purchase Month', 'Billing Month', 'ClientPM', 'Discrepancies']]

final_payment = final_payment.loc[:, ~final_payment.columns.duplicated(keep='first')]
final_payment.columns = final_payment.columns.str.replace(' ', '_')
final_details.columns = final_details.columns.str.replace(' ', '_')



#(for "print" message)number of new tickets: 
new_tickets = final_payment['Project_name'].isna().sum()
#(for "print" message)number of ENU tickets: 
count_enu = final_details[final_details['Language'] == 'CHECK']['HO_Number'].nunique()
# (for "print" message) total number of tickets: 
number_ho = final_payment['HO_Number'].count()
# (for "print" message) total number of discrepancies:
discr = final_payment['Discrepancies'].count()
print('Total number of tickets: ' + str(number_ho))
print('Number of new tickets (with On Hold status): '+str(new_tickets))
print('\nNumber of "ENU" tickets: '+str(count_enu))
print('Discrepancies: '+str(discr))



writer = pd.ExcelWriter('TEMP_FRAME.xlsx', engine='xlsxwriter') #saves the file (you can adjust file name there)
final_payment.to_excel(writer, sheet_name='PAYMENT', index=False) #index from langs_count df
final_details.to_excel(writer, sheet_name='PROJECT DETAILS', index=False)
writer.save()
writer.close()


executionTime = round((time.time() - startTime),2)
print('\nExecution time in seconds: ' + str(executionTime) + ' seconds')
final_payment

Total number of tickets: 42
Number of new tickets (with On Hold status): 3

Number of "ENU" tickets: 1
Discrepancies: 3

Execution time in seconds: 4.1 seconds


Project_name  PA#  \
0                          AWS-CMS-BF-AWSLOC32384.TRA  NaN   
1                            AWS-SW-CON-APSF.23.02.17  NaN   
2                      AWS-CMS-INT-AWSLOC31811_TransP  NaN   
3                   AWS-CMS-KB-CQClient-2023.02.23.17  NaN   
4                   AWS-CMS-KB-CQClient-2023.02.23.21  NaN   
5                            AWS-SW-CON-APSF.23.02.23  NaN   
6                    AWS-SW-CON-WAGEngine.S3.23.02.23  NaN   
7   AWS-SW-CON-AmazonConnect.LilyConsole-3e621098....  NaN   
8                  AWS-SW-CON-AWSActivate.S3.23.02.23  NaN   
9                     AWS-SW-CON-Aperture.S3.23.02.23  NaN   
10  AWS-SW-CON-CloudWatch.InternetMonitor-4fd221b2...  NaN   
11             AWS-SW-CON-SupportUI-85b889bd.23.02.23  NaN   
12                                               <NA>  NaN   
13      AWS-SW-CON-CloudWatch.Alarm-628b6684.23.02.23  NaN   
14   AWS-SW-CON-EC2.LaunchTemplates-2f5cc40d.23.02.23  NaN   
15                AWS-SW-CON-Config-97ab9768.23.02.23  NaN   
16     AWS-SW-CON-AWSMedicalImaging-9b43bbeb.23.02.23  NaN   
17                                               <NA>  NaN   
18                   AWS-SW-CON-RDS-928dc01d.23.02.23  NaN   
19  AWS-SW-CON-ResourceAccessManager-5a8ba023.23.0...  NaN   
20       AWS-SW-CON-ChainCloud-d51b57e6.23.02.23-PSCP  NaN   
21  AWS-SW-CON-AWSIETHeimdallPolidenConsole-f80757...  NaN   
22          AWS-SW-CON-Billing.llCC-5402d0b6.23.02.23  NaN   
23  AWS-SW-CON-LightSail.WebappAssets-75b55248.23....  NaN   
24         AWS-SW-CON-EMRStudio.App-429ea214.23.02.23  NaN   
25  AWS-SW-CON-ElementalMediaConnect-a032cc43.23.0...  NaN   
26            AWS-SW-CON-Transcribe-646e33fe.23.02.23  NaN   
27  AWS-SW-CON-Route53ConsoleDomainsAssets-a7ddb2b...  NaN   
28                                               <NA>  NaN   
29          AWS-SW-CON-EC2.Neutrino-04c1147e.23.02.23  NaN   
30  AWS-SW-CON-GrapheneConsoleWebapp-374d0f4b.23.0...  NaN   
31              AWS-SW-CON-Valkyrie-f19c96ad.23.02.23  NaN   
32                                 SW-CON-Aperture.S3  NaN   
33           AWS-SW-CON-HaasConsole-352f8c06.23.02.23  NaN   
34         AWS-SW-CON-SkymapConsole-0cbe9ff0.23.02.23  NaN   
35                   AWS-SW-CON-RDS-73867ca1.23.02.23  NaN   
36             AWS-SW-CON-CodeSuite-212eae7e.23.02.23  NaN   
37   AWS-SW-CON-AWSResourceExplorer-b7e38796.23.02.23  NaN   
38        AWS-SW-CON-AWSElastiCache-64b772fa.23.02.23  NaN   
39  AWS-SW-CON-SSM.AutomationAnywhere-50f9c84a.23....  NaN   
40           AWS-SW-CON-RDS.Neptune-f54f6a77.23.02.23  NaN   
41                              AWS-SW-CON-APSF.21.22  NaN   

        Client_PO_Number   HO_Number  #_of_Langs  \
0      3M-07336998 (TaC)  AWS-HO1919         8.0   
1   3M-09923940 (-APSF-)  AWS-HO1920         6.0   
2   3M-09923940 (-CORE-)  AWS-HO1921         2.0   
3     3M-09923940 (-KB-)  AWS-HO1922         9.0   
4     3M-09923940 (-KB-)  AWS-HO1923         9.0   
5   3M-09923940 (-APSF-)  AWS-HO1924         6.0   
6     3M-09923940 (-SW-)  AWS-HO1925         2.0   
7     3M-09923940 (-SW-)  AWS-HO1926        10.0   
8     3M-09923940 (-SW-)  AWS-HO1927         9.0   
9     3M-09923940 (-SW-)  AWS-HO1928        11.0   
10    3M-09923940 (-SW-)  AWS-HO1929        11.0   
11    3M-09923940 (-SW-)  AWS-HO1930         8.0   
12                  <NA>  AWS-HO1931         NaN   
13    3M-09923940 (-SW-)  AWS-HO1932        10.0   
14    3M-09923940 (-SW-)  AWS-HO1933        10.0   
15    3M-09923940 (-SW-)  AWS-HO1934        10.0   
16    3M-09923940 (-SW-)  AWS-HO1935        11.0   
17                  <NA>  AWS-HO1936         NaN   
18    3M-09923940 (-SW-)  AWS-HO1937        10.0   
19    3M-09923940 (-SW-)  AWS-HO1938        11.0   
20    3M-09923940 (-SW-)  AWS-HO1939        11.0   
21    3M-09923940 (-SW-)  AWS-HO1940         9.0   
22    3M-09923940 (-SW-)  AWS-HO1941        10.0   
23    3M-09923940 (-SW-)  AWS-HO1942        10.0   
24    3M-09923940 (-SW-)  AWS-HO1943         3.0   
25    3M-09923940 (-SW

In [5]:
Total_new

Language Translation Type    ICE  100% Matches  99-95% Matches  \
0      GER           MTPE-P  37291             3               0   
1     SPXN           MTPE-P  37291             3               0   
2      FRE           MTPE-P  37291             3               0   
3      IND           MTPE-P  37291             3               0   
4      ITA           MTPE-P  37291             3               0   
5      JPN           MTPE-P  37291             3               0   
6      KOR           MTPE-P  37291             3               0   
7      PTB           MTPE-P  37291             3               0   
8      CHS           MTPE-P  37291             3               0   
9      CHT           MTPE-P  37291             3               0   

   94-85% Matches  84-75% Matches  74-0% Matches  Reps  
0               0               0             21     0  
1               0               0             21     0  
2               0               0             21     0  
3               0               0             21     0  
4               0               0             21     0  
5               0               0             21     0  
6               0               0             21     0  
7               0               0             21     0  
8               0               0             21     0  
9               0               0             21     0